In [1]:
import sys, os
import numpy as np
import time, h5py
codepath = '/user_data/mmhender/imStat/code'
sys.path.append(codepath)
from utils import default_paths, coco_utils, nsd_utils, numpy_utils, stats_utils
from model_fitting import initialize_fitting 
from feature_extraction import texture_statistics_pyramid
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import argparse


In [2]:
subject = 1
debug = True


In [4]:
path_to_load = default_paths.sketch_token_feat_path

features_file = os.path.join(path_to_load, 'S%d_features_each_prf.h5py'%(subject))
if not os.path.exists(features_file):
    raise RuntimeError('Looking at %s for precomputed features, not found.'%features_file)   
path_to_save = os.path.join(path_to_load, 'LDA')
if not os.path.exists(path_to_save):
    os.mkdir(path_to_save)

# Params for the spatial aspect of the model (possible pRFs)
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    
n_prfs = models.shape[0]

print('Loading pre-computed features from %s'%features_file)
t = time.time()
with h5py.File(features_file, 'r') as data_set:
    values = np.copy(data_set['/features'][:,:,0:2])
    data_set.close() 
elapsed = time.time() - t
print('Took %.5f seconds to load file'%elapsed)
features_each_prf = values

zgroup_labels = np.concatenate([np.zeros(shape=(1,150)), np.ones(shape=(1,1))], axis=1)

most extreme RF positions:
[-0.55 -0.55  0.04]
[0.55       0.55       0.40000001]
Loading pre-computed features from /user_data/mmhender/features/sketch_tokens/S1_features_each_prf.h5py
Took 17.64319 seconds to load file


In [5]:
print('Size of features array for this image set is:')
print(features_each_prf.shape)

scores_each_prf = []
wts_each_prf = []
trn_acc_each_prf = []

# Gather semantic labels for the images (COCO super-categories)
coco_trn, coco_val = coco_utils.init_coco()
cat_objects, cat_names, cat_ids, supcat_names, ids_each_supcat = coco_utils.get_coco_cat_info(coco_val)
subject_df = nsd_utils.get_subj_df(subject);
all_coco_ids = np.array(subject_df['cocoId'])
ims_each_supcat = []
for sc, scname in enumerate(supcat_names):
    ims_in_supcat = coco_utils.get_ims_in_supcat(coco_trn, coco_val, scname, all_coco_ids)
    ims_each_supcat.append(ims_in_supcat)
ims_each_supcat = np.array(ims_each_supcat)
supcats_each_image = [np.where(ims_each_supcat[:,ii])[0] for ii in range(ims_each_supcat.shape[1])]

# For now, choosing just the images which have only one super-category present.
ims_to_use = [sc for sc in range(len(supcats_each_image)) if len(supcats_each_image[sc])==1]
supcat_labels = np.array([supcats_each_image[sc] for sc in ims_to_use])
n_each_supcat = [np.sum(supcat_labels==sc) for sc in np.unique(supcat_labels)]
print('\nProportion data labels each super-cat:')
print(supcat_names)
print(n_each_supcat)
print('Total images to use: %d'%np.sum(n_each_supcat))

Size of features array for this image set is:
(10000, 151, 2)
loading annotations into memory...
Done (t=0.72s)
creating index...
index created!
loading annotations into memory...
Done (t=15.59s)
creating index...
index created!

Proportion data labels each super-cat:
['accessory', 'animal', 'appliance', 'electronic', 'food', 'furniture', 'indoor', 'kitchen', 'outdoor', 'person', 'sports', 'vehicle']
[37, 1103, 87, 61, 258, 179, 249, 16, 206, 41, 31, 560]
Total images to use: 2828


In [5]:
import pandas as pd
labels_file = os.path.join(default_paths.stim_root, 'shared_1000_all_labels_matrix.csv')
labels_df = pd.read_csv(labels_file)
# sort this into order to match my subject's sequence of stimuli
df_inds = [np.where(np.array(labels_df['cocoId'])==all_coco_ids[ii])[0][0] for ii in range(1000)]
labels_df_sorted = labels_df.loc[df_inds]
labels_df_sorted = labels_df_sorted.set_index(np.arange(1000))
assert(np.all(np.array(labels_df_sorted['cocoId'])==all_coco_ids[0:1000]))
labels_df_sorted

,nsdId,cvatId,cocoId,cocoSplit,food_object_card_ratio,food_space_ratio_cmp_objects,food_space_ratio_cmp_image,food_centrality,food_relevance_caption,indoor,...,animal-face,animal-body,food,drink,food-related,faux-food,zoom,reach,large-scale-scene,object
0,2950,1,262145,train2017,0.4375,0.232896,0.045983,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,2990,2,262239,train2017,0.8000,0.385453,0.064550,0,1,1,...,0,0,1,0,1,0,0,0,0,1
2,3049,3,262414,train2017,0.0000,0.000000,0.000000,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3077,4,524646,train2017,0.0000,0.000000,0.000000,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,3146,5,262690,train2017,0.0000,0.000000,0.000000,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,72312,996,515508,train2017,0.0000,0.000000,0.000000,0,0,0,...,0,0,0,0,0,0,0,0,1,0
996,72510,997,254130,train2017,0.0000,0.000000,0.000000,0,0,0,...,0,0,0,0,0,0,0,0,1,1
997,72605,998,516634,train2017,0.0000,0.000000,0.000000,0,0,0,...,0,1,0,0,0,0,1,0,0,0
998,72719,999,304627,train2017,0.0000,0.000000,0.000000,0,0,1,...,0,0,0,0,0,0,0,0,1,1


In [31]:
# binary labels for the 16 category attributes
categ_binary = np.array(labels_df_sorted)[:,9:]
categ_binary.shape
categ_names = list(labels_df_sorted.keys())[9:]
print(categ_names)
print(categ_binary.shape)

['indoor', 'outdoor', 'ambiguous-location', 'plant', 'human-face', 'human-body', 'animal-face', 'animal-body', 'food', 'drink', 'food-related', 'faux-food', 'zoom', 'reach', 'large-scale-scene', 'object']
(1000, 16)


In [42]:
np.ones((categ_binary.shape[0],)).shape
        

(1000,)

In [134]:

# which classification schemes to do, looping over these
class_names = ['indoor_outdoor','scale','all_objects','food_face','food_object','food_vs_nofood']
categ_use_list = [[0,1],[12,13,14],[3,4,5,6,7,8,9,10,11,15], [4,8], [8, 15], [8,-1]]

n_cv = 10
debug = True

for ci, categ_use in enumerate(categ_use_list):

    print('\nClassifying based on %s'%class_names[ci])
    print(categ_use)
    if categ_use[1]==-1:
        ims_to_use = np.where(np.ones((categ_binary.shape[0],))==1)[0]
        categ_labels = categ_binary[:,categ_use[0]]==1
        print('Labels to classify between (n-way):')
        print(['no '+categ_names[categ_use[0]]] + [categ_names[categ_use[0]]])

    else:    
        ims_to_use = np.where(np.sum(categ_binary[:,categ_use],axis=1)==1)[0]
        categ_labels = np.array([np.where(categ_binary[cc,categ_use])[0][0] for cc in ims_to_use])
        print('Labels to classify between (n-way):')
        print([categ_names[cc] for cc in categ_use])

    print('unique labels:')
    print(np.unique(categ_labels))

    print('Across all %d images: proportion samples each category:'%len(ims_to_use))        
    n_each_cat = [np.sum(categ_labels==cc) for cc in np.unique(categ_labels)]
    print(np.round(n_each_cat/np.sum(n_each_cat), 2))


    n_per_cv = int(np.ceil(len(categ_labels)/n_cv))
    cv_labels = np.repeat(np.arange(n_cv), n_per_cv)
    cv_labels = cv_labels[0:len(categ_labels)]
    [np.sum(cv_labels==cv) for cv in range(n_cv)]

    class_each_prf = []

    for prf_model_index in range(n_prfs):

        if debug and prf_model_index>1:
            continue

        print('\nProcessing pRF %d of %d'%(prf_model_index, n_prfs))

        features_in_prf = features_each_prf[ims_to_use,:,prf_model_index]
        features_in_prf_z = numpy_utils.zscore_in_groups(features_in_prf, zgroup_labels)

        print('Size of features array for this image set and prf is:')
        print(features_in_prf.shape)

        pred_categ_labels = np.zeros(np.shape(categ_labels))

        for cv in range(n_cv):

            print('    cv %d of %d'%(cv, n_cv))
            trninds = cv_labels!=cv
            tstinds = cv_labels==cv

            trn_features = features_in_prf[trninds,:]
            trn_features_z = numpy_utils.zscore_in_groups(trn_features, zgroup_labels)
            tst_features = features_in_prf[tstinds,:]
            tst_features_z = numpy_utils.zscore_in_groups(tst_features, zgroup_labels)

            scores, scalings, trn_acc, trn_dprime, clf = do_lda(trn_features_z, categ_labels[trninds], \
                                                                verbose=True, balance_downsample=True)

            pred = clf.predict(tst_features_z)
            pred_categ_labels[tstinds] = pred

        print('\nFull dataset: proportion predictions each category:')        
        n_each_cat_pred = [np.sum(pred_categ_labels==cc) for cc in np.unique(categ_labels)]
        print(np.round(n_each_cat_pred/np.sum(n_each_cat_pred), 2))

        acc = np.mean(categ_labels==pred_categ_labels)
        dprime = stats_utils.get_dprime(pred_categ_labels, categ_labels)
        print('Held-out data acc = %.2f'%(acc*100))
        print('Held-out data d-prime = %.2f'%(dprime))

        class_each_prf.append({'acc': acc, 'dprime': dprime, 'actual_categ_labels': categ_labels, \
                               'pred_categ_labels': pred_categ_labels})

#     fn2save = os.path.join(path_to_save, 'S%d_LDA_preds_crossval_%s.npy'%(subject, class_names[ci]))
#     print('saving to %s'%fn2save)
#     np.save(fn2save, class_each_prf)

    


Classifying based on indoor_outdoor
[0, 1]
Labels to classify between (n-way):
['indoor', 'outdoor']
unique labels:
[0 1]
Across all 944 images: proportion samples each category:
[0.27 0.73]

Processing pRF 0 of 875
Size of features array for this image set and prf is:
(944, 151)
    cv 0 of 10
Running linear discriminant analysis: original size of array is [849 x 151]

Before downsampling - proportion samples each category:
[0.26 0.74]
Computing a new random seed
Seeding random number generator: seed is 360622

After downsampling - proportion samples each category:
[0.5 0.5]
Number of samples each category:
[222, 222]
Time elapsed: 0.02567
Training data prediction accuracy = 80.86 pct
Training data dprime = 1.76
Proportion predictions each category:
[0.47 0.53]
    cv 1 of 10
Running linear discriminant analysis: original size of array is [849 x 151]

Before downsampling - proportion samples each category:
[0.26 0.74]
Computing a new random seed
Seeding random number generator: seed 

In [133]:
values = np.concatenate([np.ones((100,15)), 2*np.ones((100,15))], axis=0)
values = values+np.random.normal(0,1,np.shape(values))*1
categ_labels = np.repeat(np.arange(2), 100)
# np.random.shuffle(categ_labels)
trninds = np.arange(0,180)
tstinds = np.arange(180,200)
scores, scalings, trn_acc, trn_dprime, clf = do_lda(values[trninds,:], categ_labels[trninds], verbose=True)
pred_tst = clf.predict(values[tstinds,:])
# pred_tst
np.mean(pred_tst==categ_labels[tstinds])


Running linear discriminant analysis: original size of array is [180 x 15]

Before downsampling - proportion samples each category:
[0.56 0.44]
Computing a new random seed
Seeding random number generator: seed is 470522

After downsampling - proportion samples each category:
[0.5 0.5]
Number of samples each category:
[80, 80]
Time elapsed: 0.00306
Training data prediction accuracy = 98.75 pct
Training data dprime = 4.48
Proportion predictions each category:
[0.5 0.5]


0.9

In [107]:
  
def do_lda(values, categ_labels, verbose=False, balance_downsample=True, rndseed=None):
    """
    Apply linear discriminant analysis to the data - find axes that best separate the given category labels.
    """
    n_features_actual = values.shape[1]
    n_trials = values.shape[0]
    n_each_cat = [np.sum(categ_labels==cc) for cc in np.unique(categ_labels)]
    if verbose:        
        print('Running linear discriminant analysis: original size of array is [%d x %d]'%(n_trials, \
                                                                                           n_features_actual))
    
    # Balance class labels    
    min_samples = np.min(n_each_cat)
    if not np.all(n_each_cat==min_samples) and balance_downsample:
        
        if verbose:
            print('\nBefore downsampling - proportion samples each category:')               
            print(np.round(n_each_cat/np.sum(n_each_cat), 2)) 

        un = np.unique(categ_labels)
        if rndseed is None:
            if verbose:
                print('Computing a new random seed')
            shuff_rnd_seed = int(time.strftime('%S%M%H', time.localtime()))
        if verbose:
            print('Seeding random number generator: seed is %d'%shuff_rnd_seed)
        np.random.seed(shuff_rnd_seed)
        samples_to_use = []
        for ii in range(len(un)):
            samples_to_use += [np.random.choice(np.where(categ_labels==un[ii])[0], min_samples, replace=False)]
        samples_to_use = np.array(samples_to_use).ravel()
        
        values = values[samples_to_use,:]
        categ_labels = categ_labels[samples_to_use]
        n_each_cat = [np.sum(categ_labels==cc) for cc in np.unique(categ_labels)]
    
        if verbose:
            print('\nAfter downsampling - proportion samples each category:')               
            print(np.round(n_each_cat/np.sum(n_each_cat), 2)) 
            print('Number of samples each category:')
            print(n_each_cat) 
        
    elif verbose:       
        print('\nClasses are balanced - proportion samples each category:')        
        print(np.round(n_each_cat/np.sum(n_each_cat), 2)) 
        
    t = time.time()
    
    X = values; y = np.squeeze(categ_labels)
    clf = LinearDiscriminantAnalysis(solver='svd')
    clf.fit(X, y)

    elapsed = time.time() - t
    if verbose:
        print('Time elapsed: %.5f'%elapsed)
    
    ypred = clf.predict(X)
    trn_acc = np.mean(y==ypred)
    trn_dprime = stats_utils.get_dprime(ypred, y)
    if verbose:
        print('Training data prediction accuracy = %.2f pct'%(trn_acc*100))
        print('Training data dprime = %.2f'%(trn_dprime))
        print('Proportion predictions each category:')        
        n_each_cat_pred = [np.sum(ypred==cc) for cc in np.unique(categ_labels)]
        print(np.round(n_each_cat_pred/np.sum(n_each_cat_pred), 2))

    # Transform method is doing : scores = (X - clf.xbar_) @ clf.scalings_
    # So the "scalings" define a mapping from mean-centered feature data to the LDA subspace.
    scores = clf.transform(X)    
    scalings = clf.scalings_
   
    return scores, scalings, trn_acc, trn_dprime, clf

